In [273]:
import pandas as pd
import numpy as np
import requests

In [274]:
portfolio = pd.read_csv('portfolio.csv')
portfolio['Дата'] = portfolio['Дата'].astype('datetime64[ns]')
transactions = pd.read_csv('transactions.csv').drop_duplicates(['Дата заключения', 'Время заключения', 'Статус'])
transactions['Дата заключения'] = pd.to_datetime(transactions['Дата заключения'] + ' ' + transactions['Время заключения'], dayfirst=True)
transactions['Дата расчетов'] = pd.to_datetime(transactions['Дата расчетов'], dayfirst=True)
transactions = transactions.drop('Время заключения', axis=1).dropna(axis=1).sort_values('Дата заключения').reset_index(drop=True)

In [275]:
transactions

,Дата заключения,Дата расчетов,Наименование,Код,Валюта,Вид,Количество,Цена,Сумма,НКД,Комиссия Брокера,Комиссия Биржи,Номер сделки,Статус
0,2021-05-17 10:21:16,2021-05-19,SBSPETF,SBSP,RUB,Покупка,1,1633.000,1633.0,0.0,0.98,0.15,3943134134,О
1,2021-05-17 10:21:16,2021-05-19,SBSPETF,SBSP,RUB,Покупка,1,1633.000,1633.0,0.0,0.98,0.15,3943134134,И
2,2021-05-17 10:21:16,2021-05-19,SBSPETF,SBSP,RUB,Покупка,1,1633.000,1633.0,0.0,0.98,0.15,3943134134,ЗО
3,2021-05-17 10:22:48,2021-05-19,FXUSETF,FXUS,RUB,Покупка,1,5615.000,5615.0,0.0,3.37,0.52,3943142919,И
4,2021-05-17 10:22:48,2021-05-19,FXUSETF,FXUS,RUB,Покупка,1,5615.000,5615.0,0.0,3.37,0.52,3943142919,О
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,2023-06-23 10:33:59,2023-06-27,МТС-ао,MTSS,RUB,Покупка,10,343.350,3433.5,0.0,2.06,1.03,7792729150,О
263,2023-07-18 16:07:57,2023-07-20,СевСт-ао,CHMF,RUB,Покупка,1,1275.600,1275.6,0.0,0.77,0.38,7908972071,И
264,2023-07-18 17:52:50,2023-07-20,Сбербанк,SBER,RUB,Покупка,10,246.210,2462.1,0.0,1.48,0.73,7909781928,И
265,2023-07-18 17:55:20,2023-07-20,Системаао,AFKS,RUB,Покупка,100,17.744,1774.4,0.0,1.06,0.54,7909796018,И


In [276]:
transactions_executed = transactions[transactions['Статус'] == 'И']

In [277]:
ticker_list = list(transactions_executed['Код'].unique())
share_split_dict = {
    'FXGD': ['2022-02-17', 10],
    'SBMX': ['2021-06-09', 100],
    'FXUS': ['2022-01-24', 100],
    'FXRL': ['2021-11-24', 100],
    'FXRU': ['2022-02-17', 10],
    'FXDE': ['2021-12-15', 100]
}
replacement_dict = {
    'VTBE': 'RSHE',
    'RU000A101FA1': 'SU25084RMFS3'
}

In [278]:
def fix_split():
    ticker_list_copy = ticker_list.copy()
    for ticker in ticker_list_copy:
        if ticker == 'RU000A101FA1':
            new_ticker = 'SU25084RMFS3'
            ticker_list.remove(ticker)
            ticker_list.append(new_ticker)
            transactions_executed.loc[transactions['Код'] == ticker, 'Код'] = new_ticker
        if ticker == 'VTBE':
            new_ticker = 'RSHE'
            ticker_list.remove(ticker)
            ticker_list.append(new_ticker)
            transactions_executed.loc[transactions['Код'] == ticker, 'Код'] = new_ticker
    for ticker in ticker_list:
        if ticker in share_split_dict:
            transactions_executed.loc[(transactions_executed['Код'] == ticker) & (transactions_executed['Дата заключения'] < share_split_dict[ticker][0]), 'Количество'] = \
            transactions_executed.loc[(transactions_executed['Код'] == ticker) & (transactions_executed['Дата заключения'] < share_split_dict[ticker][0]), 'Количество'] * \
            share_split_dict[ticker][1]
fix_split()

In [279]:
def get_share_price_dict(tickers: list):
    share_price_dict = {}
    for ticker in tickers:
        if ticker in ['SBMX', 'AFKS']:
            round_numb = 3
        elif ticker in ['IRAO']:
            round_numb = 4
        else:
            round_numb = 2
        share = transactions_executed[transactions_executed['Код'] == ticker]
        share_price = ((share['Сумма'].sum() + share['Комиссия Брокера'].sum() + share['Комиссия Биржи'].sum()) / 
                            (share.loc[share['Вид'] == 'Покупка', 'Количество'].sum() - share.loc[share['Вид'] == 'Продажа', 'Количество'].sum())).round(round_numb)
        share_price_dict[ticker] = share_price
    return share_price_dict

In [280]:
share_price_dict = get_share_price_dict(ticker_list)

In [281]:
def get_share_amount_dict(tickers: list):
    share_amount_dict = {}
    for ticker in tickers:
        share_amount_dict[ticker] = transactions_executed.loc[transactions_executed['Код'] == ticker, 'Количество'].sum()
    return share_amount_dict

In [282]:
share_amount_dict = get_share_amount_dict(ticker_list)

In [283]:
def get_trading_dict():
    trading_dict = {}
    trading_mode_list = ['TQBR', 'TQTF', 'TQCB', 'TQOB']
    for id_trading in trading_mode_list:
        stocks = 'shares'
        if id_trading in ['TQOB', 'TQCB']:
            stocks = 'bonds'
        url = (f"https://iss.moex.com/iss/engines/stock/markets/{stocks}/boards/{id_trading}/"
            f"securities.csv?iss.meta=off&iss.only=marketdata&marketdata.columns=SECID,LAST")
        csv_text = requests.get(url).text.split('\n')
        trading_dict[id_trading] = csv_text
    return trading_dict

In [284]:
def get_last_prices_dict(tickers: list):
    last_prices_dict = {}
    trading_dict = get_trading_dict()
    for ticker in tickers:
        if ticker == 'RU000A101FA1':
            ticker =  'SU25084RMFS3'
        if ticker == 'VTBE':
            ticker = 'RSHE'
        for value in trading_dict.values():
            for line in value:
                line = line.split(';')
                if ticker in line:
                    last_prices_dict[ticker] = line[1]
    return last_prices_dict

In [285]:
last_prices_dict = get_last_prices_dict(ticker_list)

In [286]:
def get_commission_dict(tickers: list):
    commission_dict = {}
    for ticker in tickers:
        commission_dict[ticker] = (transactions_executed.loc[transactions_executed['Код'] == ticker, 'Комиссия Брокера'] + \
                                     transactions_executed.loc[transactions_executed['Код'] == ticker, 'Комиссия Биржи']) \
                                    .sum().round(2)
    return commission_dict

In [287]:
commission_dict = get_commission_dict(ticker_list)

In [288]:
def get_coupon_dict(tickers: list):
    coupon_dict = {}
    url = "https://iss.moex.com/iss/engines/stock/markets/bonds/boards/TQOB/securities.csv?iss.meta=off&iss.only" \
          "=securities&securities.columns=SECID,ACCRUEDINT "
    csv_text = requests.get(url).text.split('\n')

    for ticker in tickers:
        for line in csv_text:
            line = line.split(';')
            if ticker in line:
                coupon_dict[ticker] = line[1]
    return coupon_dict

In [289]:
coupon_dict = get_coupon_dict(ticker_list)

In [290]:
portfolio_dict = {
    'Котировки': last_prices_dict,
    'НКД': coupon_dict,
    'Количество': share_amount_dict,
    'Средняя цена': share_price_dict,
    'Комиссия': commission_dict
}

main = pd.DataFrame.from_dict(portfolio_dict)
main.index.name = 'Название'
main.loc[main['Котировки'] == '', 'Котировки'] = np.nan
main.dropna(axis=0, subset='Котировки', inplace=True)
main[['Котировки', 'НКД']] = main[['Котировки', 'НКД']].astype('float64')
# main.loc['SU25084RMFS3', 'Котировки'] = main.loc['SU25084RMFS3', 'Котировки'] * 10
# main.loc['SU25084RMFS3', 'Текущая цена'] = (main.loc['SU25084RMFS3', 'Котировки'] + main.loc['SU25084RMFS3', 'НКД']) * main.loc['SU25084RMFS3', 'Количество']
main['Текущая цена'] = main['Котировки'] * main['Количество']
main['P/L, руб.'] = main['Текущая цена'] - main['Средняя цена'] * main['Количество']
main['P/L, %'] = (main['Котировки'] * 100 / main['Средняя цена'] - 100).round(2)

In [291]:
bond_price, bond_NKD, bond_amount, bond_average_price, bond_commission = \
            main.loc['SU25084RMFS3', 'Котировки'], \
            main.loc['SU25084RMFS3', 'НКД'], \
            main.loc['SU25084RMFS3', 'Количество'], \
            main.loc['SU25084RMFS3', 'Средняя цена'], \
            main.loc['SU25084RMFS3', 'Комиссия']

In [292]:
main

,Котировки,НКД,Количество,Средняя цена,Комиссия,Текущая цена,"P/L, руб.","P/L, %"
Название,,,,,,,,
SBMX,17.7050,NaN,830,15.7570,4.98,14695.150,1616.840,12.36
SBER,264.1500,NaN,40,224.6600,8.06,10566.000,1579.600,17.58
MTSS,289.4000,NaN,20,300.9000,5.41,5788.000,-230.000,-3.82
GAZP,172.8900,NaN,30,174.1700,4.71,5186.700,-38.400,-0.73
LKOH,6146.0000,NaN,1,4821.3300,4.33,6146.000,1324.670,27.48
MGNT,5612.0000,NaN,1,4186.2600,3.76,5612.000,1425.740,34.06
YNDX,2717.0000,NaN,1,2642.3800,2.38,2717.000,74.620,2.82
CHMF,1339.2000,NaN,1,1276.7500,1.15,1339.200,62.450,4.89
AFKS,18.2940,NaN,100,17.7600,1.60,1829.400,53.400,3.01
